# Concepts
## Dosing Ladder


## Rule
A rule has a `parameter`, `operation`, and a `threshold` such that running the `operation` on `parameter` and `threshold` always returns a boolean value (True or False). This value can be computed by running the method `is_satisfied`.
The inner state of a `Rule` should be independent of any one patient.

## Action
A specific recommendation or action. It has three methods that can be defined for each specific action type: the first returns a text version, the second a clickable button, and the third implements what should be done when the button is clicked. `Action`s are specific to each patient.

## Rule With Actions
Extends `Rule` but also suggests a list of `Action`s depending on whether it is satisfied or not. These are defined by two attributes: `actions_when_satisfied` and `actions_when_not_satisfied`. In addition to `Action`s, these two attributes can hold other Rules with Actions. The method `get_suggested_actions` evaluates the `Rule` and selects the appropriate list of actions. If the list contains other Rules With Actions, these are evaluated recursively until the list only contains `Action`s.

## Titrator
The actual engine running the titration process. Has `checkpoints`, which is a list of `Rule(s)WithActions`. `checkpoints` indicates both initiation and titration rules, but these can be specified separately by defining `inititation_checkpoints` and `titration_checkpoints`.
`Titrator`s are specific to each patient and medication class.

In [ ]:
#|export
from typing import List, Dict, Any
from titrations.basics import *

# Dosing Ladder

In [ ]:
#|export

class DosingLadder:
    ladder : Dict[str, List[Medication]]

    def __init__(self, ladder_dict : Dict[str, List[Medication]], single_class : bool = True) -> None:
        # TODO: ensure 'subladders' have the same number of steps

        # TODO: ensure each subladder consists of the same ingredient

        if single_class:
            pass # TODO: ensure all medications are of the same class

        self.ladder = ladder_dict
        # self.ingredient = None
        self.med_class = None

    @property
    def ingredients(self) -> List[Ingredient]:
        # TODO this should be okay if the checks in `__init__` are implemented
        return [self.ladder[med_name][0].ingredient for med_name in self.ladder]

    def get_subladder(self, medication : Ingredient | Medication):
        return self.ladder[medication.name]

    def _get_current_step_index(self, current_med : Medication):
        subladder = self.get_subladder(current_med)
        index = next((i for i, med in enumerate(subladder) if med.dose == current_med.dose), None)
        return index
    
    def _is_at_lowest_step(self, current_med : Medication):
        return current_med.dose == self.get_lowest_step(current_med).dose

    def _is_at_highest_step(self, current_med : Medication):
        return current_med.dose == self.get_highest_step(current_med).dose
    
    def get_next_step_up(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index + 1]

    def get_next_step_down(self, current_med : Medication):
        current_dose_index = self._get_current_step_index(current_med)
        return self.get_subladder(current_med)[current_dose_index - 1]
    
    def get_lowest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[0]

    def get_highest_step(self, medication: Ingredient | Medication):
        return self.get_subladder(medication)[-1]
    
    def get_current_medication_for_patient(self, patient : Patient):
        filtered = list(filter(lambda med: med.ingredient in self.ingredients, patient.medications))
        assert len(filtered) <= 1
        return filtered[0] if filtered else None
    
    @property
    def lowest_steps(self):
        return { med_name : self.ladder[med_name][0] for med_name in self.ladder}
    
    @property
    def highest_steps(self):
        return { med_name : self.ladder[med_name][-1] for med_name in self.ladder}

In [ ]:
metoprolol_succinate = Ingredient("metoprolol succinate")
carvedilol = Ingredient("carvedilol")
bisoprolol = Ingredient("bisoprolol")

In [ ]:
beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

In [ ]:
beta_blocker_ladder.get_subladder(metoprolol_succinate)

In [ ]:
beta_blocker_ladder.get_highest_step(metoprolol_succinate)

In [ ]:
beta_blocker_ladder.get_lowest_step(metoprolol_succinate)

In [ ]:
beta_blocker_ladder._get_current_step_index(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
    )

In [ ]:
beta_blocker_ladder._is_at_lowest_step(
    Medication(metoprolol_succinate, "12.5 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder._is_at_highest_step(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.get_next_step_up(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.get_next_step_down(
    Medication(metoprolol_succinate, "25 mg", "PO", "daily")
)

In [ ]:
beta_blocker_ladder.lowest_doses

In [ ]:
beta_blocker_ladder.highest_doses

In [ ]:
p = Patient(medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")])
beta_blocker_ladder.get_current_medication_for_patient(p)

# Rules

In [ ]:
#|export
import operator

class Rule:
    parameter : str
    operation : str
    threshold : Any

    operators = {
        "gt": operator.gt,
        "gte": operator.ge,
        "lt": operator.lt,
        "lte": operator.le,
        "eq": operator.eq,
        "neq": operator.ne,
        "in": operator.contains,
    }

    def __init__(self, parameter:str, operation:str, threshold:Any) -> None:
        assert parameter in VALID_PARAMETERS, "Not a valid parameter"
        assert operation in self.operators, f"Invalid operator {operation}"

        self.parameter = parameter
        self.operation = operation
        self.threshold = threshold

    def _is_satisfied(self, patient : Patient):
        patient_value = getattr(patient, self.parameter, None)
        if patient_value is None:  # FIXME: this should be reviewed again
            if type(self.threshold) == bool: raise ValueError(f"Patient has no attribute `{self.parameter}`.")
            else: return False

        if self.operation == "in": return self.operators[self.operation](self.threshold, patient_value)
        else: return self.operators[self.operation](patient_value, self.threshold)

    def _get_eval_result_object(self, is_satisfied : bool) -> Any:
        return type('RuleEvalResult', (), {
            'is_satisfied': is_satisfied,
        })

    def evaluate(self, patient : Patient):
        is_satisfied = self._is_satisfied(patient)
        return self._get_eval_result_object(is_satisfied)
    
    def __repr__(self) -> str:
        return f"{self.parameter} {self.operation} {self.threshold}"

In [ ]:
r = Rule('SBP', 'lt', 100)
eval_result = r.evaluate(Patient(SBP=100))
eval_result.is_satisfied

In [ ]:
#|export

class ConditionalRule(Rule):
    condition : Rule

    def __init__(self, parameter:str, operation:str, threshold:Any, condition:Rule) -> None:
        super().__init__(parameter, operation, threshold)
        self.condition = condition

    def _condition_is_met(self, patient: Patient):
        return self.condition.evaluate(patient).is_satisfied

    def _is_satisfied(self, patient: Patient, condition_is_met: bool):
        return super()._is_satisfied(patient) if condition_is_met else False

    def evaluate(self, patient: Patient):
        condition_is_met = self._condition_is_met(patient)
        is_satisfied = self._is_satisfied(patient, condition_is_met)
        return self._get_eval_result_object(is_satisfied)


In [ ]:
no_pacemaker = Rule('has_pacemaker', 'eq', False)

In [ ]:
p = Patient(HR=50, has_pacemaker=False)
result = no_pacemaker.evaluate(p)
result.is_satisfied

In [ ]:
bradycardia = ConditionalRule('HR', 'lt', 60, condition=no_pacemaker)
p = Patient(HR=80, has_pacemaker=False)
bradycardia.evaluate(p).is_satisfied

# Actions

In [ ]:
#|export

class Action:
    """
    This is a base class for 'Action' classes to build upon.
    """
    def __init__(self,
                 patient : Patient,
                 dosing_ladder : DosingLadder,
                 current_medication : Medication):
        self.patient = patient
        self.dosing_ladder = dosing_ladder
        self.current_medication = current_medication

    def suggest(self):
        pass

    def perform(self):
        pass

In [ ]:
#|export

class Start(Action):
    """
    Start a new medication.
    """
    def suggest(self):
        lowest_steps = self.dosing_ladder.lowest_steps
        meds = list(lowest_steps)
        # TODO: this should only be done if the ladder has 3+ subladders
        return f"Start {', '.join([str(lowest_steps[med]) for med in meds[:-1]])}, or {str(lowest_steps[meds[-1]])}."

    def perform(self):
        pass

In [ ]:
a = Start(p, beta_blocker_ladder)
a.suggest()

In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}
list(d)

In [ ]:
#|export

class DoNotStart(Action):
    """
    Do not start a new medication.
    """
    def suggest(self):
        # TODO: modify to suggest not starting a class
        pass

    def perform(self):
        pass

In [ ]:
#|export

class StepUp(Action):
    """
    Step up one dose on the dosing ladder.
    """
    def suggest(self):
        next_step_up = self.dosing_ladder.get_next_step_up(self.current_medication)
        return f"Increase {self.current_medication.name} to {str(next_step_up)}."

    def perform(self):
        pass

In [ ]:
#|export

class StepDown(Action):
    """
    Step down one dose on the dosing ladder.
    """
    def suggest(self):
        next_step_down = self.dosing_ladder.get_next_step_down(self.current_medication)
        return f"Decrease {self.current_medication.name} to {str(next_step_down)}."

    def perform(self):
        pass

In [ ]:
#|export

class Continue(Action):
    """
    Continue the medication at the same dose.
    """
    def suggest(self):
        return f"Continue {str(self.current_medication)}."

    def perform(self):
        pass

In [ ]:
#|export

class Stop(Action):
    """
    Stop the medication.
    """
    def suggest(self):
        return f"Stop {self.current_medication.name}."

    def perform(self):
        pass

In [ ]:
#|export

class MarkMaxDose(Action):
    """
    Mark current dose as maxiumum tolerated dose.
    """
    def suggest(self):
        return f"Mark {str(self.current_medication)} as maximum tolerated dose."
    
    def perform(self):
        pass

In [ ]:
#|export
class ReportReaction(Action):
    """
    File an adverse reaction record.
    """
    def suggest(self):
        return f"File an adverse reaction to {self.current_medication.name}"

    def perform(self, description="reaction"):
        # In real life, this could be opening a modal for entering adverse reactions instead
        self.patient.reactions.append(
            Reaction(self.current_medication.ingredient, description),
        )

# Rules With Actions

In [ ]:
#|export

class RuleWithActions(Rule):
    actions_when_satisfied : List[Any] = []
    actions_when_not_satisfied : List[Any] = []
    
    def __init__(self,
                 parameter: str, operation: str, threshold: Any,
                 additional_actions_when_satisfied: List[Any] = [],
                 additional_actions_when_not_satisfied: List[Any] = []) -> None:
        super().__init__(parameter, operation, threshold)
        self.actions_when_satisfied.extend(additional_actions_when_satisfied)
        self.actions_when_not_satisfied.extend(additional_actions_when_not_satisfied)

    def _get_eval_result_object(self, is_satisfied: bool) -> Any:
        result = super()._get_eval_result_object(is_satisfied)

        recommended_actions = self.actions_when_satisfied if is_satisfied \
            else self.actions_when_not_satisfied
        result.recommended_actions = recommended_actions
        return result

class ConditionalRuleWithActions(ConditionalRule, RuleWithActions):
    pass

In [ ]:
r = RuleWithActions('SBP', 'lt', 90,
                    additional_actions_when_satisfied=['Evaluate for hypotension'],
                    additional_actions_when_not_satisfied=['Continue'])
p = Patient(SBP=60)
r.evaluate(p).__dict__

# Titrator

In [ ]:
#|export
class Titrator:
    patient : Patient
    dosing_ladder : DosingLadder

    current_medication : Medication
    
    titration_target : Rule

    rules : List[RuleWithActions]
    initiation_rules : List[RuleWithActions]
    titration_rules = List[RuleWithActions]

    def __init__(self, patient : Patient,
                 dosing_ladder : DosingLadder) -> None:
        self.patient = patient
        self.dosing_ladder = dosing_ladder

        self.current_medication = self.dosing_ladder.get_current_medication_for_patient(patient)

    @property
    def current_ingredient(self) -> Ingredient:
        return self.current_medication.ingredient if self.current_medication else None

    @property
    def is_initiating(self) -> bool:
        return self.current_medication == None

    @property
    def is_titrating(self) -> bool:
        return not self.is_initiating


In [ ]:
t = Titrator(p, beta_blocker_ladder)

In [ ]:
t.current_ingredient

In [ ]:
t.is_initiating

In [ ]:
t.is_titrating